In [0]:
#UPLOAD
from google.colab import files
uploaded = files.upload()

Saving submission.csv to submission.csv
Saving test.csv to test.csv
Saving train.csv to train.csv


In [0]:
ls

kaggle.json  sample_data/  submission.csv  test.csv  train.csv


In [0]:
#IMPORT LIBRARIES
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from scipy import stats
from scipy.stats import norm, skew
from xgboost import XGBClassifier
from matplotlib import dates


#CONFIG
color = sns.color_palette()
sns.set_style('darkgrid')

In [0]:
#TRAINING AND TESTING THE DATA
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
submission = pd.read_csv('submission.csv')
train_data.head()

,CustomerID,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,CountryRegionName,PostalCode,PhoneNumber,BirthDate,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,AveMonthSpend,BikeBuyer
0,22409,NaN,Candace,NaN,Madan,NaN,1488 Guadalupe,NaN,Hervey Bay,Queensland,Australia,4655,1 (11) 500 555-0115,8/15/1979,Bachelors,Clerical,F,S,1,0,0,0,11336,37,0
1,15809,NaN,Christian,NaN,Sharma,NaN,6086 Glen Wood Drive,NaN,Port Macquarie,New South Wales,Australia,2444,1 (11) 500 555-0159,9/15/1960,Partial College,Skilled Manual,M,M,1,3,4,5,81978,135,1
2,27510,NaN,Cole,Q,Bailey,NaN,6253 Panorama Dr.,NaN,Burlingame,California,United States,94010,350-555-0145,10/3/1976,Partial College,Professional,M,S,0,2,0,0,86892,70,0
3,17627,NaN,Dennis,M,Chen,NaN,3195 Rotherham Dr.,NaN,Maidenhead,England,United Kingdom,SL67RJ,1 (11) 500 555-0181,9/9/1974,Bachelors,Skilled Manual,M,M,1,0,1,1,68021,90,1
4,21238,NaN,Carl,NaN,Tang,NaN,516 Oak Park Blvd.,NaN,Cliffside,British Columbia,Canada,V8Y 1L1,386-555-0142,9/25/1980,High School,Skilled Manual,M,S,0,2,0,0,52525,64,1


In [0]:
#TRAINING AND TESTING THE DATA
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
submission = pd.read_csv('submission.csv')
train_data.head().T

,0,1,2,3,4
CustomerID,22409,15809,27510,17627,21238
Title,NaN,NaN,NaN,NaN,NaN
FirstName,Candace,Christian,Cole,Dennis,Carl
MiddleName,NaN,NaN,Q,M,NaN
LastName,Madan,Sharma,Bailey,Chen,Tang
Suffix,NaN,NaN,NaN,NaN,NaN
AddressLine1,1488 Guadalupe,6086 Glen Wood Drive,6253 Panorama Dr.,3195 Rotherham Dr.,516 Oak Park Blvd.
AddressLine2,NaN,NaN,NaN,NaN,NaN
City,Hervey Bay,Port Macquarie,Burlingame,Maidenhead,Cliffside
StateProvinceName,Queensland,New South Wales,California,England,British Columbia


In [0]:
test_data.head()

,CustomerID,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,CountryRegionName,PostalCode,PhoneNumber,BirthDate,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,AveMonthSpend,BikeBuyer
0,25140,NaN,Toni,K,Lopez,NaN,6445 Cashew Street,NaN,Rockhampton,Queensland,Australia,4700,1 (11) 500 555-0176,3/29/1964,Bachelors,Professional,F,M,1,2,3,3,86912,NaN,1
1,25844,NaN,Riley,M,Butler,NaN,3271 Norse Ct,NaN,Coronado,California,United States,92118,293-555-0134,10/19/1940,High School,Professional,F,S,0,2,0,4,55516,NaN,0
2,25352,NaN,Brandon,S,Martinez,NaN,843 Camino Verde Cr.,NaN,Lake Oswego,Oregon,United States,97034,327-555-0193,2/14/1957,Partial High School,Clerical,M,M,0,2,1,2,18570,NaN,0
3,26332,NaN,Aaron,W,Mitchell,NaN,3773 Lakefield Place,NaN,Lebanon,Oregon,United States,97355,116-555-0185,9/26/1966,Bachelors,Skilled Manual,M,M,0,0,3,3,121120,NaN,1
4,25987,NaN,Hunter,NaN,Kumar,NaN,222 Hunter Drive,NaN,Bellflower,California,United States,90706,475-555-0151,5/25/1962,High School,Skilled Manual,M,M,1,2,2,3,55513,NaN,1


**DATA EXPLORATION**

In [0]:
print('Training Data', train_data.shape)
print('Test Data', test_data.shape)

Training Data (7654, 25)
Test Data (3340, 25)


In [0]:
train_data.describe()

,CustomerID,Suffix,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,AveMonthSpend,BikeBuyer
count,7654.000000,0.0,7654.000000,7654.000000,7654.000000,7654.000000,7654.000000,7654.000000,7654.000000
mean,18784.735824,NaN,0.695192,1.581657,1.253201,2.161092,82289.196891,76.505357,0.498171
std,4795.026146,NaN,0.460356,1.186209,1.659555,1.731424,40658.245901,29.465195,0.500029
min,11001.000000,NaN,0.000000,0.000000,0.000000,0.000000,10036.000000,22.000000,0.000000
25%,14760.750000,NaN,0.000000,1.000000,0.000000,1.000000,50765.750000,53.000000,0.000000
50%,18479.500000,NaN,1.000000,2.000000,0.000000,2.000000,82233.500000,72.000000,0.000000
75%,22431.500000,NaN,1.000000,2.000000,2.000000,4.000000,109853.500000,89.000000,1.000000
max,29481.000000,NaN,1.000000,4.000000,5.000000,5.000000,193721.000000,176.000000,1.000000


In [0]:
test_data.isnull().any(), train_data.isnull().any()

(CustomerID              False
 Title                    True
 FirstName               False
 MiddleName               True
 LastName                False
 Suffix                   True
 AddressLine1            False
 AddressLine2             True
 City                    False
 StateProvinceName       False
 CountryRegionName       False
 PostalCode              False
 PhoneNumber             False
 BirthDate               False
 Education               False
 Occupation              False
 Gender                  False
 MaritalStatus           False
 HomeOwnerFlag           False
 NumberCarsOwned         False
 NumberChildrenAtHome    False
 TotalChildren           False
 YearlyIncome            False
 AveMonthSpend            True
 BikeBuyer               False
 dtype: bool, CustomerID              False
 Title                    True
 FirstName               False
 MiddleName               True
 LastName                False
 Suffix                   True
 AddressLine1            F

In [0]:
train_data['Title'].unique()

array([nan, 'Mr.', 'Ms.', 'Sr.', 'Mrs.'], dtype=object)

In [0]:
# train_data['Title'].fillna(train_data['Title'].mode()[0], inplace=True)
# test_data['Title'].fillna(test_data['Title'].mode()[0], inplace=True)

train_data['Title'].unique()

array([nan, 'Mr.', 'Ms.', 'Sr.', 'Mrs.'], dtype=object)

In [0]:
#Abrreviating Words
train_data.head(2)

,CustomerID,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,CountryRegionName,PostalCode,PhoneNumber,BirthDate,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,AveMonthSpend,BikeBuyer
0,22409,NaN,Candace,NaN,Madan,NaN,1488 Guadalupe,NaN,Hervey Bay,Queensland,Australia,4655,1 (11) 500 555-0115,8/15/1979,Bachelors,Clerical,F,S,1,0,0,0,11336,37,0
1,15809,NaN,Christian,NaN,Sharma,NaN,6086 Glen Wood Drive,NaN,Port Macquarie,New South Wales,Australia,2444,1 (11) 500 555-0159,9/15/1960,Partial College,Skilled Manual,M,M,1,3,4,5,81978,135,1


In [0]:
train_data.select_dtypes(exclude=['int', 'float']).columns

Index(['Title', 'FirstName', 'MiddleName', 'LastName', 'AddressLine1',
       'AddressLine2', 'City', 'StateProvinceName', 'CountryRegionName',
       'PostalCode', 'PhoneNumber', 'BirthDate', 'Education', 'Occupation',
       'Gender', 'MaritalStatus'],
      dtype='object')

In [0]:
from datetime import datetime

def calculate_age(born):
    datetime_obj = datetime.strptime(born, '%m/%d/%Y')
    today = datetime.today()
    return today.year - datetime_obj.year - ((today.month, today.day) < (datetime_obj.month, datetime_obj.day))

In [0]:
print (calculate_age('8/15/1993'))

26


In [0]:
train_data.head(3)

,CustomerID,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,CountryRegionName,PostalCode,PhoneNumber,BirthDate,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,AveMonthSpend,BikeBuyer
0,22409,NaN,Candace,NaN,Madan,NaN,1488 Guadalupe,NaN,Hervey Bay,Queensland,Australia,4655,1 (11) 500 555-0115,8/15/1979,Bachelors,Clerical,F,S,1,0,0,0,11336,37,0
1,15809,NaN,Christian,NaN,Sharma,NaN,6086 Glen Wood Drive,NaN,Port Macquarie,New South Wales,Australia,2444,1 (11) 500 555-0159,9/15/1960,Partial College,Skilled Manual,M,M,1,3,4,5,81978,135,1
2,27510,NaN,Cole,Q,Bailey,NaN,6253 Panorama Dr.,NaN,Burlingame,California,United States,94010,350-555-0145,10/3/1976,Partial College,Professional,M,S,0,2,0,0,86892,70,0


In [0]:
train_data['Age'] = train_data['BirthDate'].apply(calculate_age)
test_data['Age'] = test_data['BirthDate'].apply(calculate_age)

In [0]:
#dropping BirthDate
train_data.drop('BirthDate', axis = 1, inplace = True)
test_data.drop('BirthDate', axis = 1, inplace = True)

In [0]:
train_data.head(3)

,CustomerID,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,CountryRegionName,PostalCode,PhoneNumber,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,AveMonthSpend,BikeBuyer,Age
0,22409,NaN,Candace,NaN,Madan,NaN,1488 Guadalupe,NaN,Hervey Bay,Queensland,Australia,4655,1 (11) 500 555-0115,Bachelors,Clerical,F,S,1,0,0,0,11336,37,0,40
1,15809,NaN,Christian,NaN,Sharma,NaN,6086 Glen Wood Drive,NaN,Port Macquarie,New South Wales,Australia,2444,1 (11) 500 555-0159,Partial College,Skilled Manual,M,M,1,3,4,5,81978,135,1,59
2,27510,NaN,Cole,Q,Bailey,NaN,6253 Panorama Dr.,NaN,Burlingame,California,United States,94010,350-555-0145,Partial College,Professional,M,S,0,2,0,0,86892,70,0,43


In [0]:
#dropping AddressLine
train_data.drop('AddressLine1', axis = 1, inplace = True)
test_data.drop('AddressLine1', axis = 1, inplace = True)

In [0]:
#dropping AddressLine2
train_data.drop('AddressLine2', axis = 1, inplace = True)
test_data.drop('AddressLine2', axis = 1, inplace = True)

In [0]:
#dropping Names
train_data.drop('FirstName', axis = 1, inplace = True)
test_data.drop('FirstName', axis = 1, inplace = True)

train_data.drop('MiddleName', axis = 1, inplace = True)
test_data.drop('MiddleName', axis = 1, inplace = True)

In [0]:
#dropping Suffix
train_data.drop('Suffix', axis = 1, inplace = True)
test_data.drop('Suffix', axis = 1, inplace = True)

#dropping Title
train_data.drop('Title', axis = 1, inplace = True)
test_data.drop('Title', axis = 1, inplace = True)

train_data.drop('PostalCode', axis = 1, inplace = True)
test_data.drop('PostalCode', axis = 1, inplace = True)

train_data.drop('PhoneNumber', axis = 1, inplace = True)
test_data.drop('PhoneNumber', axis = 1, inplace = True)


In [0]:
train_data.head(3)

,CustomerID,LastName,City,StateProvinceName,CountryRegionName,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,AveMonthSpend,BikeBuyer,Age
0,22409,Madan,Hervey Bay,Queensland,Australia,Bachelors,Clerical,F,S,1,0,0,0,11336,37,0,40
1,15809,Sharma,Port Macquarie,New South Wales,Australia,Partial College,Skilled Manual,M,M,1,3,4,5,81978,135,1,59
2,27510,Bailey,Burlingame,California,United States,Partial College,Professional,M,S,0,2,0,0,86892,70,0,43


In [0]:
train_data['Education'].value_counts()

Bachelors              2337
Partial College        2064
High School            1337
Graduate Degree        1316
Partial High School     600
Name: Education, dtype: int64

In [0]:
train_data['Occupation'].value_counts()

Professional      2489
Skilled Manual    1808
Management        1327
Clerical          1077
Manual             953
Name: Occupation, dtype: int64

**FEATURE ENGINEERING**

In [0]:
test_data.head()

,CustomerID,LastName,City,StateProvinceName,CountryRegionName,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,AveMonthSpend,BikeBuyer,Age
0,25140,Lopez,Rockhampton,Queensland,Australia,Bachelors,Professional,F,M,1,2,3,3,86912,NaN,1,55
1,25844,Butler,Coronado,California,United States,High School,Professional,F,S,0,2,0,4,55516,NaN,0,79
2,25352,Martinez,Lake Oswego,Oregon,United States,Partial High School,Clerical,M,M,0,2,1,2,18570,NaN,0,62
3,26332,Mitchell,Lebanon,Oregon,United States,Bachelors,Skilled Manual,M,M,0,0,3,3,121120,NaN,1,53
4,25987,Kumar,Bellflower,California,United States,High School,Skilled Manual,M,M,1,2,2,3,55513,NaN,1,57


In [0]:
test_data['AveMonthSpend'].unique()

array([nan])

In [0]:

test_data.drop('AveMonthSpend', axis = 1, inplace = True)

In [0]:
train_data.describe()

,CustomerID,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,AveMonthSpend,BikeBuyer,Age
count,7654.000000,7654.000000,7654.000000,7654.000000,7654.000000,7654.000000,7654.000000,7654.000000,7654.000000
mean,18784.735824,0.695192,1.581657,1.253201,2.161092,82289.196891,76.505357,0.498171,56.947870
std,4795.026146,0.460356,1.186209,1.659555,1.731424,40658.245901,29.465195,0.500029,10.539022
min,11001.000000,0.000000,0.000000,0.000000,0.000000,10036.000000,22.000000,0.000000,39.000000
25%,14760.750000,0.000000,1.000000,0.000000,1.000000,50765.750000,53.000000,0.000000,49.000000
50%,18479.500000,1.000000,2.000000,0.000000,2.000000,82233.500000,72.000000,0.000000,56.000000
75%,22431.500000,1.000000,2.000000,2.000000,4.000000,109853.500000,89.000000,1.000000,63.000000
max,29481.000000,1.000000,4.000000,5.000000,5.000000,193721.000000,176.000000,1.000000,108.000000


In [0]:
def formatYearlyIncome(income):
  if(income > 0  and income <= 20000):
    return 1
  if(income > 20000  and income <= 40000):
    return 2
  if(income > 40000  and income <= 60000):
    return 3
  if(income > 60000  and income <= 80000):
    return 4
  if(income > 80000  and income <= 100000):
    return 5
  if(income > 100000  and income <= 120000):
    return 6
  if(income > 120000  and income <= 140000):
    return 7
  if(income > 140000  and income <= 160000):
    return 8
  if(income > 160000):
    return 9

In [0]:
train_data['formatYearlyIncome'] = train_data['YearlyIncome'].apply(formatYearlyIncome)
test_data['formatYearlyIncome'] = test_data['YearlyIncome'].apply(formatYearlyIncome)

In [0]:
train_data.head()

,CustomerID,LastName,City,StateProvinceName,CountryRegionName,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,AveMonthSpend,BikeBuyer,Age,formatYearlyIncome
0,22409,Madan,Hervey Bay,Queensland,Australia,Bachelors,Clerical,F,S,1,0,0,0,11336,37,0,40,1
1,15809,Sharma,Port Macquarie,New South Wales,Australia,Partial College,Skilled Manual,M,M,1,3,4,5,81978,135,1,59,5
2,27510,Bailey,Burlingame,California,United States,Partial College,Professional,M,S,0,2,0,0,86892,70,0,43,5
3,17627,Chen,Maidenhead,England,United Kingdom,Bachelors,Skilled Manual,M,M,1,0,1,1,68021,90,1,45,4
4,21238,Tang,Cliffside,British Columbia,Canada,High School,Skilled Manual,M,S,0,2,0,0,52525,64,1,39,3


In [0]:
train_data.drop('YearlyIncome', axis = 1, inplace = True)
test_data.drop('YearlyIncome', axis = 1, inplace = True)

In [0]:
test_data.head()

,CustomerID,LastName,City,StateProvinceName,CountryRegionName,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,BikeBuyer,Age,formatYearlyIncome
0,25140,Lopez,Rockhampton,Queensland,Australia,Bachelors,Professional,F,M,1,2,3,3,1,55,5
1,25844,Butler,Coronado,California,United States,High School,Professional,F,S,0,2,0,4,0,79,3
2,25352,Martinez,Lake Oswego,Oregon,United States,Partial High School,Clerical,M,M,0,2,1,2,0,62,1
3,26332,Mitchell,Lebanon,Oregon,United States,Bachelors,Skilled Manual,M,M,0,0,3,3,1,53,7
4,25987,Kumar,Bellflower,California,United States,High School,Skilled Manual,M,M,1,2,2,3,1,57,3


In [0]:
test_data.describe()

,CustomerID,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,BikeBuyer,Age,formatYearlyIncome
count,3340.000000,3340.000000,3340.000000,3340.000000,3340.000000,3340.000000,3340.000000,3340.000000
mean,23441.067964,0.625150,1.545509,1.182036,2.120958,0.495210,56.976347,4.563473
std,5107.867886,0.484157,1.183150,1.632014,1.703996,0.500052,10.381595,2.048641
min,11026.000000,0.000000,0.000000,0.000000,0.000000,0.000000,39.000000,1.000000
25%,20374.250000,0.000000,1.000000,0.000000,1.000000,0.000000,49.000000,3.000000
50%,25180.500000,1.000000,2.000000,0.000000,2.000000,0.000000,56.000000,5.000000
75%,27368.250000,1.000000,2.000000,2.000000,4.000000,1.000000,64.000000,6.000000
max,29480.000000,1.000000,4.000000,5.000000,5.000000,1.000000,101.000000,9.000000


In [0]:
def formatAge(age):
  if(age > 35  and age <= 40):
    return 1
  if(age > 40  and age <= 45):
    return 2
  if(age > 45  and age <= 50):
    return 3
  if(age > 50  and age <= 55):
    return 4
  if(age > 55  and age <= 60):
    return 5
  if(age > 60  and age <= 65):
    return 6
  if(age > 65  and age <= 70):
    return 7
  if(age > 70  and age <= 75):
    return 8
  if(age > 75):
    return 9

In [0]:
train_data['formatedAge'] = train_data['Age'].apply(formatAge)
test_data['formatedAge'] = test_data['Age'].apply(formatAge)

In [0]:
train_data.drop('Age', axis = 1, inplace = True)
test_data.drop('Age', axis = 1, inplace = True)

In [0]:
train_data.head(5)

,CustomerID,LastName,City,StateProvinceName,CountryRegionName,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,AveMonthSpend,BikeBuyer,formatYearlyIncome,formatedAge
0,22409,Madan,Hervey Bay,Queensland,Australia,Bachelors,Clerical,F,S,1,0,0,0,37,0,1,1
1,15809,Sharma,Port Macquarie,New South Wales,Australia,Partial College,Skilled Manual,M,M,1,3,4,5,135,1,5,5
2,27510,Bailey,Burlingame,California,United States,Partial College,Professional,M,S,0,2,0,0,70,0,5,2
3,17627,Chen,Maidenhead,England,United Kingdom,Bachelors,Skilled Manual,M,M,1,0,1,1,90,1,4,2
4,21238,Tang,Cliffside,British Columbia,Canada,High School,Skilled Manual,M,S,0,2,0,0,64,1,3,1


In [0]:
categorical = [
 'LastName',
 'City',
 'StateProvinceName',
 'CountryRegionName',
 'Education',
 'Occupation',
 'Gender',
 'MaritalStatus']

In [0]:
#LabelEncoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for column in categorical :
    train_data[column] = le.fit_transform(train_data[column])
    

for column in categorical :
    test_data[column] = le.fit_transform(test_data[column])

In [0]:
train_data.head()

,CustomerID,LastName,City,StateProvinceName,CountryRegionName,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,AveMonthSpend,BikeBuyer,formatYearlyIncome,formatedAge
0,22409,121,101,29,0,0,0,0,1,1,0,0,0,37,0,1,1
1,15809,221,178,21,0,3,4,1,0,1,3,4,5,135,1,5,5
2,27510,13,31,5,5,3,3,1,1,0,2,0,0,70,0,5,2
3,17627,42,133,7,4,0,4,1,0,1,0,1,1,90,1,4,2
4,21238,247,56,4,1,2,4,1,1,0,2,0,0,64,1,3,1


In [0]:
test_data.head()

,CustomerID,LastName,City,StateProvinceName,CountryRegionName,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,BikeBuyer,formatYearlyIncome,formatedAge
0,25140,111,172,24,0,0,3,0,0,1,2,3,3,1,5,4
1,25844,29,50,5,5,2,3,0,1,0,2,0,4,0,3,9
2,25352,118,99,22,5,4,0,1,0,0,2,1,2,0,1,6
3,26332,123,106,22,5,0,4,1,0,0,0,3,3,1,7,4
4,25987,101,3,5,5,2,4,1,0,1,2,2,3,1,3,5


In [0]:
train_data.head()

,LastName,City,StateProvinceName,CountryRegionName,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,AveMonthSpend,BikeBuyer,formatYearlyIncome,formatedAge
0,121,101,29,0,0,0,0,1,1,0,0,0,37,0,1,1
1,221,178,21,0,3,4,1,0,1,3,4,5,135,1,5,5
2,13,31,5,5,3,3,1,1,0,2,0,0,70,0,5,2
3,42,133,7,4,0,4,1,0,1,0,1,1,90,1,4,2
4,247,56,4,1,2,4,1,1,0,2,0,0,64,1,3,1


In [0]:
train_data_num = train_data['CustomerID']
test_data_num = test_data['CustomerID']

train_data.drop("CustomerID", axis = 1, inplace = True)
test_data.drop("CustomerID", axis = 1, inplace = True)

In [0]:
#save and drop the target varriable
y_train=train_data['AveMonthSpend']
train_data.drop('AveMonthSpend',axis=1,inplace=True)

In [0]:
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split


x_trn,x_valid, y_trn, y_valid = train_test_split(train_data,y_train, test_size = 0.2, random_state = 42)

In [0]:
#Applying XGboost
model_xg =XGBClassifier(objective="binary:logistic", 
    learning_rate=0.1, 
    min_child_weight=7,
    max_depth=7, 
    gamma=0, 
    n_estimators=200)
model_xg.fit(train_data,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.3, max_delta_step=0, max_depth=7,
              min_child_weight=7, missing=None, n_estimators=200, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [0]:
#Our Score on Validation Data
model_xg.score(x_valid,y_valid)

0.9653821032005225

In [0]:
#Prediction on Test Set
predict_y_xg = model_xg.predict(test_data)

In [0]:
#Our Score on Training data
model_xg.score(x_trn,y_trn)

0.9547607382002287

In [0]:
submission.columns

Index(['CustomerID', 'AveMonthSpend'], dtype='object')

In [0]:
#Prepare our Submission file
my_submission = pd.DataFrame({'CustomerID':test_data_num, 'AveMonthSpend': predict_y_xg })
my_submission.to_csv('submission_team_02.csv', index=False)


In [0]:
ls

kaggle.json   submission.csv          test.csv
sample_data/  submission_team_02.csv  train.csv


In [0]:
files.download('submission_team_02.csv')